In [1]:
import pandas as pd
import json

In [6]:
dat = json.load(open("db_out.json"))['experiment2']['participantData']

with open('../public/verifiable_all_data.json') as f:
    txt = "".join(f.readlines())[7:]
questions = json.loads(txt)

In [7]:
col_names = ['firebase_id','prolific_pid','session_id','sentence_set_id','total_time_taken','question_id',
            'question_index','statement_id','statement_text','name','gender','uncertainty_expression','response','time_taken']
results = []

In [8]:
for p in dat.keys():
    results_p = []
    db_p = dat[p]
    results_p.extend([
        p, db_p['PROLIFIC_PID'], db_p['SESSION_ID'], db_p['sentenceSetId'], db_p['totalTimeTaken']*0.001
    ])
    for q in db_p['responses'].keys():
        if q[:2] != "p-":    
            results_pq = results_p.copy()
            q_num = q.split('-')[1]
            q_info = questions[db_p['sentenceSetId']][q_num]
            p_info = db_p['responses'][q]
            results_pq.extend([
                q_num,
                p_info['index'],
                q_info['statement_id'],
                q_info['prompt_example'],
                q_info['name'],
                q_info['gender'],
                q_info['uncertainty_expression'],
                int(p_info['response']),
                p_info['timeTaken']*0.001
            ])
            results.append(results_pq)

In [9]:
df = pd.DataFrame(results, columns=col_names)
df = df.sort_values(by=['session_id','question_index']).reset_index(drop=True)
print("{} participants included in dataframe".format(len(df['session_id'].unique())))

99 participants included in dataframe


In [11]:
df.to_csv('humans_experiment2.csv', index=False)

In [16]:
col_names_s = ['firebase_id','uncertainty_expression','index','response']

In [17]:
sanity_checks = []
for p in dat.keys():
    results_p = []
    db_p = dat[p]
    results_p.append(p)
    for q in ['p-'+str(i) for i in range(5)]:
        results_pq = results_p.copy()
        p_info = db_p['responses'][q]
        results_pq.extend([
            p_info['phrase'],
            p_info['index'],
            p_info['response'],
        ])
        sanity_checks.append(results_pq)

In [18]:
pd.DataFrame(sanity_checks, columns=col_names_s).to_csv('sanity_checks.csv', index=False)

In [19]:
df.to_csv('humans_experiment1.csv', index=False)